In [7]:
import tensorflow as tf
from tensorflow import keras

In [8]:
tf.__version__

'2.15.0'

In [10]:
!wget -nc https://github.com/DataTalksClub/machine-learning-zoomcamp/releases/download/chapter7-model/xception_v4_large_08_0.894.h5 -O clothing-model.h5

File ‘clothing-model.h5’ already there; not retrieving.


In [11]:
!python -V

Python 3.10.13


# Load the model

In [12]:
model = keras.models.load_model('clothing-model.h5')

In [13]:
# Download picture

In [15]:
!wget -nc http://bit.ly/mlbookcamp-pants -O pants.jpg

File ‘pants.jpg’ already there; not retrieving.


In [16]:
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np

In [17]:
img = load_img('pants.jpg', target_size=(299,299))

In [18]:
X = np.array(img)

In [19]:
X = preprocess_input(X[tf.newaxis, ...])

In [20]:
preds = model.predict(X)

1/1 [==============================] - 0s 359ms/step


In [21]:
preds

array([[-1.879864 , -4.756312 , -2.3595333, -1.0892622,  9.903783 ,
        -2.82618  , -3.64831  ,  3.241155 , -2.6120963, -4.852036 ]],
      dtype=float32)

In [22]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

In [23]:
dict(zip(classes, preds[0]))


{'dress': -1.879864,
 'hat': -4.756312,
 'longsleeve': -2.3595333,
 'outwear': -1.0892622,
 'pants': 9.903783,
 'shirt': -2.82618,
 'shoes': -3.64831,
 'shorts': 3.241155,
 'skirt': -2.6120963,
 't-shirt': -4.852036}

# Convert Keras to TF-Lite

In [51]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('clothing-model.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /var/folders/xp/nfq4xfg90wzfc3w1r68x1hkr0000gp/T/tmpsl__082a/assets


INFO:tensorflow:Assets written to: /var/folders/xp/nfq4xfg90wzfc3w1r68x1hkr0000gp/T/tmpsl__082a/assets
2023-11-25 06:21:12.569745: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-25 06:21:12.569757: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-25 06:21:12.569893: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/xp/nfq4xfg90wzfc3w1r68x1hkr0000gp/T/tmpsl__082a
2023-11-25 06:21:12.579876: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-25 06:21:12.579884: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /var/folders/xp/nfq4xfg90wzfc3w1r68x1hkr0000gp/T/tmpsl__082a
2023-11-25 06:21:12.607744: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-25 06:21:12.901235: I tensorflow/cc/saved_model/loader.cc:217] Running initialization

In [2]:
import tensorflow.lite as tflite

In [3]:
interpreter = tflite.Interpreter(model_path='clothing-model.tflite')
interpreter.allocate_tensors()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [5]:
interpreter.get_signature_list()

{'serving_default': {'inputs': ['input_8'], 'outputs': ['dense_7']}}

In [26]:
predictions_lite = interpreter.get_signature_runner('serving_default')(input_8=X)['dense_7']
score_lite = tf.nn.softmax(predictions_lite)

In [29]:
dict(zip(classes, score_lite.numpy()[0]))


{'dress': 7.6182255e-06,
 'hat': 4.2917034e-07,
 'longsleeve': 4.7156077e-06,
 'outwear': 1.6796124e-05,
 'pants': 0.99868613,
 'shirt': 2.957161e-06,
 'shoes': 1.299657e-06,
 'shorts': 0.0012761031,
 'skirt': 3.6631166e-06,
 't-shirt': 3.899927e-07}

In [30]:
print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(classes[np.argmax(score_lite)], 100 * np.max(score_lite))
)

This image most likely belongs to pants with a 99.87 percent confidence.


In [32]:
print(np.max(np.abs(preds - predictions_lite)))


2.861023e-06


### Check how to set the inputs and where to get the outputs from

In [33]:
interpreter.get_input_details()

[{'name': 'serving_default_input_8:0',
  'index': 0,
  'shape': array([  1, 299, 299,   3], dtype=int32),
  'shape_signature': array([ -1, 299, 299,   3], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [34]:
interpreter.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 229,
  'shape': array([ 1, 10], dtype=int32),
  'shape_signature': array([-1, 10], dtype=int32),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [36]:
input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

### Initialize the network with the input data

In [37]:
interpreter.set_tensor(input_index, X)

In [38]:
# initialize calculations

interpreter.invoke()

In [39]:
# get predictions

lite_preds = interpreter.get_tensor(output_index)